In [3]:
# Import necessary libraries
import numpy as np
import gradio as gr
from tensorflow.keras.models import load_model
import os
import warnings
warnings.filterwarnings('ignore')

# Define paths
models_path = '../models/'

# Load the trained Neural Network model
model = load_model(os.path.join(models_path, 'neural_net_tuned.h5'))

# Define meaningful column names, ranges, and default values
feature_details = {
    'Kidney_Disease_Severity': {'range': (0, 9), 'default': 0, 'step': 1},
    'High_Blood_Pressure': {'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'High_Cholesterol': {'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Cholesterol_Check': {'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Asthma_Severity': {'range': (0, 9), 'default': 0, 'step': 1},
    'COPD_Severity': {'range': (0, 9), 'default': 0, 'step': 1},
    'Body_Mass_Index': {'range': (0, 60), 'default': 25, 'step': 0.1},
    'Smoker': {'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Stroke_History': {'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Heart_Disease_or_Attack': {'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Physical_Activity': {'options': [0, 1], 'default': 1, 'info': '0 = No, 1 = Yes'},
    'Heavy_Alcohol_Consumption': {'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Has_Healthcare': {'options': [0, 1], 'default': 1, 'info': '0 = No, 1 = Yes'},
    'No_Doctor_Due_to_Cost': {'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'General_Health_Score': {'range': (0, 5), 'default': 2, 'step': 1},
    'Mental_Health_Days': {'range': (0, 30), 'default': 0, 'step': 1},
    'Physical_Health_Days': {'range': (0, 30), 'default': 0, 'step': 1},
    'Difficulty_Walking': {'options': [0, 1], 'default': 0, 'info': '0 = No, 1 = Yes'},
    'Sex': {'options': [0, 1], 'default': 0, 'labels': ['Female', 'Male'], 'info': '0 = Female, 1 = Male'},
    'Age_Group': {'range': (1, 13), 'default': 7, 'step': 1},
    'Education_Level': {'range': (1, 6), 'default': 3, 'step': 1},
    'Income_Level': {'range': (1, 11), 'default': 6, 'step': 1}
}

# Prediction label mapping
label_map = {0: 'No Diabetes', 1: 'Slightly has Diabetes', 2: 'Has Diabetes'}

# Validation function
def validate_input(**kwargs):
    for key, value in kwargs.items():
        if key in feature_details:
            details = feature_details[key]
            if 'range' in details and (value < details['range'][0] or value > details['range'][1]):
                return f"Invalid {key}: Value must be between {details['range'][0]} and {details['range'][1]}"
            elif 'options' in details and value not in details['options']:
                return f"Invalid {key}: Value must be one of {details['options']}"
    return None

# Prediction function
def predict_diabetes(
    Kidney_Disease_Severity, High_Blood_Pressure, High_Cholesterol, Cholesterol_Check,
    Asthma_Severity, COPD_Severity, Body_Mass_Index, Smoker, Stroke_History,
    Heart_Disease_or_Attack, Physical_Activity, Heavy_Alcohol_Consumption, Has_Healthcare,
    No_Doctor_Due_to_Cost, General_Health_Score, Mental_Health_Days, Physical_Health_Days,
    Difficulty_Walking, Sex, Age_Group, Education_Level, Income_Level
):
    # Validate inputs
    validation_error = validate_input(
        Kidney_Disease_Severity=Kidney_Disease_Severity, High_Blood_Pressure=High_Blood_Pressure,
        High_Cholesterol=High_Cholesterol, Cholesterol_Check=Cholesterol_Check,
        Asthma_Severity=Asthma_Severity, COPD_Severity=COPD_Severity, Body_Mass_Index=Body_Mass_Index,
        Smoker=Smoker, Stroke_History=Stroke_History, Heart_Disease_or_Attack=Heart_Disease_or_Attack,
        Physical_Activity=Physical_Activity, Heavy_Alcohol_Consumption=Heavy_Alcohol_Consumption,
        Has_Healthcare=Has_Healthcare, No_Doctor_Due_to_Cost=No_Doctor_Due_to_Cost,
        General_Health_Score=General_Health_Score, Mental_Health_Days=Mental_Health_Days,
        Physical_Health_Days=Physical_Health_Days, Difficulty_Walking=Difficulty_Walking,
        Sex=Sex, Age_Group=Age_Group, Education_Level=Education_Level, Income_Level=Income_Level
    )
    if validation_error:
        return validation_error

    # Prepare input data (placeholder for your preprocessing)
    # Assuming 48 features after one-hot encoding or similar preprocessing
    # Adjust this based on your preprocessor.joblib or training data structure
    input_data = np.array([
        Kidney_Disease_Severity, High_Blood_Pressure, High_Cholesterol, Cholesterol_Check,
        Asthma_Severity, COPD_Severity, Body_Mass_Index, Smoker, Stroke_History,
        Heart_Disease_or_Attack, Physical_Activity, Heavy_Alcohol_Consumption, Has_Healthcare,
        No_Doctor_Due_to_Cost, General_Health_Score, Mental_Health_Days, Physical_Health_Days,
        Difficulty_Walking, Sex, Age_Group, Education_Level, Income_Level,
        # Placeholder zeros for remaining features
        *[0] * (48 - 22)  # Adjust 48 based on your total feature count
    ]).reshape(1, -1)

    # Make prediction
    prediction_proba = model.predict(input_data)
    prediction = np.argmax(prediction_proba, axis=1)[0]
    confidence = prediction_proba[0, prediction]

    # Return result
    return f"Prediction: {label_map[prediction]} (Confidence: {confidence:.2f})"

# Create Gradio interface
inputs = [
    gr.Slider(minimum=v['range'][0], maximum=v['range'][1], step=v.get('step', 1), value=v['default'], label=k)
    if 'range' in v else gr.Dropdown(choices=v['options'], value=v['default'], label=k, info=v.get('info', ''))
    for k, v in feature_details.items()
]

interface = gr.Interface(
    fn=predict_diabetes,
    inputs=inputs,
    outputs="text",
    title="Diabetes Prediction Tool",
    description="Enter your health and demographic details to predict diabetes status. Output will be 'No Diabetes', 'Slightly has Diabetes', or 'Has Diabetes' with a confidence score."
)

# Launch the interface
interface.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
